In [27]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

from IPython.display import display, HTML
from notebook.services.config import ConfigManager

display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [28]:
""" Load sheet and change column type to string and data to uppercase """

cf_sheet = pd.read_excel('../cf/CF_OLD.xlsx').filter(["CC", "A", "B", "C", "D", "E", "F"])

other_sheet = pd.read_excel('./acko.xlsx').filter(["CC", "A", "B", "C", "D", "E", "F", "TYPE"])
other_sheet = other_sheet[ other_sheet["TYPE"] == "car" ]

for col in ["A", "B", "C", "D", "E"]:
    cf_sheet[ col ] = cf_sheet[ col ].astype("string")
    other_sheet[ col ] = other_sheet[ col ].astype("string")
    cf_sheet[ col ] = cf_sheet[ col ].str.upper()
    cf_sheet[ col ] = cf_sheet[ col ].str.strip()
    other_sheet[ col ] = other_sheet[ col ].str.upper()
    other_sheet[ col ] = other_sheet[ col ].str.strip()

other_sheet["E"]  = other_sheet["E"].replace(np.nan, "0")

cf_sheet["D"]  = cf_sheet["D"].replace( "INTERNAL_LPG_CNG", "CNG")
cf_sheet["D"]  = cf_sheet["D"].replace( "ELECTRICITY", "ELECTRIC")

other_sheet["D"]  = other_sheet["D"].replace( "INTERNAL_LPG_CNG", "CNG")

cf_sheet_non_edited = cf_sheet.copy()
other_sheet_non_edited = other_sheet.copy()

In [29]:
# cf_sheet
# other_sheet

In [30]:
""" COMMENT LINES BELOW, Used to filter only Makes in list """

MAKES =  [ "HONDA", "HYUNDAI", "KIA", "MAHINDRA", "MARUTI", "TATA", "TOYOTA" ]

cf_sheet = cf_sheet[ cf_sheet["A"].isin(MAKES) == True ]
other_sheet = other_sheet[ other_sheet["A"].isin(MAKES) == True ]

# cf_sheet["A"].unique()
# other_sheet["A"].unique()

In [31]:
""" Split model name and add to variant name """

MODEL_SPLIT_LIST = {
    "3 SERIES CONVERTIBLE": ["3", "CONVERTIBLE"],
    "A8 L": ["A8", "L"],
    "ALTURAS G4": ["ALTURAS", "G4"],
    "AZURE T": ["AZURE", "T"],
    "ACTIVE I20": ["I20", "ACTIVE"],
    "BIG NEW WAGONR": ["WAGONR", ""],
    "BOLERO NEO": ["BOLERO", "NEO"],
    "BOLERO POWER PLUS": ["BOLERO", "POWER PLUS"],
    "CITY ZX": ["CITY", "ZX"],
    "CAYENNE GTS": ["CAYENNE","GTS"],
    "CAYENNE S": ["CAYENNE","S"],
    "CAYENNE TURBO": ["CAYENNE", "TURBO"],
    "CAYENNE TURBO S": ["CAYENNE", "TURBO S"],
    "CAYMAN S": ["CAYMAN", "S"],
    "CONTINENTAL GT": ["CONTINENTAL", "GT"],
    "ESTILO 1.0  NEW": ["ESTILO", ""],
    "ELITE I20": ["I20", "ELITE"],
    "FUSION PLUS": ["FUSION", "PLUS"],
    "FIESTA15": ["FIESTA", ""],
    "FIESTA CLASSIC": ["FIESTA", "CLASSIC"],
    "FLUIDIC VERNA": ["VERNA", "FLUIDIC"],
    "FLUIDIC VERNA 4S": ["VERNA", "FLUIDIC 4S"],
    "HILUX SURF": ["HILUX", "SURF"],
    "I10 1.2 KAPPA": ["I10", "KAPPA"],
    "I20 N LINE": ["I20", "N LINE"],
    "INDICA V2 DICOR": ["INDICA", "V2 DICOR"],
    "INDICA V2 TURBO": ["INDICA", "V2 TURBO"],
    "INDICA EV2 CR4 (DIESEL)": ["INDICA EV2", ""],
    "INNOVA CRYSTA": ["INNOVA", "CRYSTA"],
    "LAND CRUISER 200": ["LAND CRUISER", "200"],
    "NANO GENX": ["NANO", "GENX"],
    "NEW AMAZE": ["AMAZE", ""],
    "NEW CRETA": ["CRETA", ""],
    "NEW ELITE I20": ["I20", "ELITE"],
    "NEW ERTIGA": ["ERTIGA", ""],
    "NEW GRAND I10": ["GRAND I10", ""],
    "I20 (2020)": ["NEW I20", ""],
    "NEW VERNA": ["VERNA", ""],
    "NEXON EV": ["NEXON", "EV"],
    "PETRA DIESEL": ["PETRA", "DIESEL"],
    "PALIO STILE": ["PALIO", "STILE"],
    "PALIO STILE MULTIJET": ["PALIO", "STILE MULTIJET"],
    "PUNTO AVVENTURA": ["PUNTO","AVVENTURA"],
    "PUNTOEVO": ["PUNTO", "EVO"],
    "PANAMERA 4": ["PANAMERA", "4"],
    "PANAMERA S": ["PANAMERA", "S"],
    "PANAMERA 4S": ["PANAMERA", "4S"],
    "PANAMERA TURBO": ["PANAMERA", "TURBO"],
    "POLO GT": ["POLO", "GT"],
    "RS6 AVANT": ["RS6", "AVANT"],
    "SCORPIO GETAWAY": ["SCORPIO", "GETAWAY"],
    "SCORPIO 2.0": ["SCORPIO", ""],
    "SCORPIO CRDE": ["SCORPIO", "CRDE"],
    "SCORPIO W CRDE": ["SCORPIO", "W CRDE"],
    "SCORPIO M2DICR": ["SCORPIO", "M2DICR"],
    "SIENA WEEKEND": ["SIENA", "WEEKEND"],
    "SWIFT 2018": ["SWIFT", "2018"],
    "SUNNY XTRONIC": ["SUNNY", "XTRONIC"],
    "TATA MANZA": ["MANZA", ""],
    "THAR CRDE": ["THAR", "CRDE"],
    "THAR DI": ["THAR", "DI"],
    "TIAGO JTP": ["TIAGO", "JTP"],
    "TIAGO NRG": ["TIAGO", "NRG"],
    "TIGOR EV": ["TIGOR", ""],
    "TIGOR JTP": ["TIGOR", "JTP"],
    "TT COUPE": ["TT", "COUPE"],
    "TUV300 PLUS": ["TUV300", "PLUS"],
    "URBAN CRUISER": ["URBAN", "CRUISER"],
    "URBAN CRUISER HYRYDER": ["URBAN", "CRUISER HYRYDER"],
    "VANTAGE V8": ["VANTAGE", "V8"],
    "VERNA TRANSFORM": ["VERNA", "TRANSFORM"],
    "VITARA BREZZA": ["VITARA", "BREZZA"],
    "V40 CROSS COUNTRY": ["V40", "CROSS COUNTRY" ],
    "WAGON R 1.0": ["WAGONR", ""],
    "WAGON R DUO": ["WAGONR", "DUO"],
    "WAGONR K10": ["WAGONR", "K10"],
    "X4 M SPORT X": ["X4", "M SPORT X"],
    "ZEN ESTILO": ["ZEN", "ESTILO"],
}

for model in MODEL_SPLIT_LIST:
    variants = cf_sheet[ cf_sheet["B"] == model ]
    for index, row in variants.iterrows():
        cf_sheet.loc[index, "B"] = MODEL_SPLIT_LIST[model][0]
        cf_sheet.loc[index, "C"] = f'{MODEL_SPLIT_LIST[model][1]} {row["C"]}'

for model in MODEL_SPLIT_LIST:
    variants = other_sheet[ other_sheet["B"] == model ]
    for index, row in variants.iterrows():
        other_sheet.loc[index, "B"] = MODEL_SPLIT_LIST[model][0]
        other_sheet.loc[index, "C"] = f'{MODEL_SPLIT_LIST[model][1]} {row["C"]}'  

# for index, row in pb_sheet[ other_sheet["A"] == "MERCEDES-BENZ" ].iterrows():
#     other_sheet.loc[index, "B"] = row["B"].replace( r"CLASS", "")
# for index, row in cf_sheet[ cf_sheet["A"] == "MERCEDES-BENZ" ].iterrows():
#     cf_sheet.loc[index, "B"] = row["B"].replace( r"CLASS", "")

In [32]:
for col in ["A", "B", "C", "D", "E"]:
    cf_sheet[col] = cf_sheet[col].str.replace("-"," ", regex=True)
    other_sheet[col] = other_sheet[col].str.replace("-"," ", regex=True)
    
MAKE_CHANGE = { r'\s?(COOPER|MOTORS|MOTORS|SUZUKI|AND\sMAHINDRA)\s?': '' }

MODEL_CHANGE = {
    "\.|\s+|1.5|1.1|SERIES|BMW|ELECTRIC|\(PETROL\)": " ",
    "\s": "",
    "WAGONRSTINGRAY": "STINGRAY",
    "LANDCRUSIERPRADO": "LANDCRUISERPRADO",
    "FLUIDICELANTRA": "ELANTRAFLUIDIC",
    "ARNAGE": "ARANGE",
    "MICRAMC": "MICRA",
    "MINICOOPER": "",
    "I20ACTIVE": "ACTIVEI20",
    "CARONA": "CORONA",
    "SIERRA": "SIERA",
}

VARIANT_CHANGE = {
    r'^\d\s?\.\d{1,2}\s?[L]?$|[LDP]?\d\.\d{1,2}\s?[L]?\s+|\d\.\d{1,2}|\d\.\d{1,2}$': " ",
    r'\d{2,3}\s?B?HP': " ",
    r'\(\s?\d{4}\s*CC\s?\)': " ",
    r'^\(\s?4\s?\+\s?1\s?/\)\s': " 5 SEATER ",
    r'\s?\+\s?LUXURY\s': ' LUXURY ',
    r'\s?\+\s?INNOVATION\s': ' INNOVATION ',
    r'\+': " PLUS ",
    r'\(O\)\s': " OPTIONAL ",
    r'\(|\)|\|': " ",
    r'STR$|\s?STR\s|\sSTR$|\sSEAT\s|SEATRE': " SEATER ",
    r'\sANN\s' : " ANNIVERSARY ",
    
    "DULE|DUEL": "DUAL",
    "PRIVLIGE": "PRIVILEGE",
    "SPETROL": "S PETROL",
    "SDIESEL": "S DIESEL",
    "AMBTION": " AMBITION ",
    "AMBIENCE": " AMBIENTE ",
    "TITATNIUM": " TITANIUM ",
    r'SPORT[SZ]': ' SPORT ',
    " PANAROMA SUNROOF ": " PANAROMA ",
    r'XDRIVE\s': " XDRIVE",
    "STEPWAY": "STEPAWAY",
    "COUPÉ": " COUPE ",
    "REVOTROQ": "REVOTORQ",
    "PRIVILAGE": "PRIVILEGE",
    
    r'\s\dS\s?(EATER)?': " ",
    r'\d\s*SEATER': " ",
    
    "CVT AUTOMATIC": "CVT",

    r'4\s?\*\s?4': ' 4WD ',
    r'4\s?\*\s?2': ' 2WD ',
    r'4\s?X\s?2': ' 2WD ',
    r'4\s?X\s?4': ' 4WD ',
    "L\s?&\s?K": " LNK ",
    "INTERNAL_LPG_CNG|A?LPG|CNG": " ",
    
    "^A\s|\sA\s": " A",
    "^B\s|\sB\s": " A",
    "^C\s|\sC\s": " C",
    "^R\s|\sR\s": " R",
    
    r'^SMT' : " S MT ",
#     r'\sI\s': "I ",
    r'\sLD\s': "LD ",
    r'\sLI\s': "LI ",
    "ATGARDEE": " ATGARDE E ",
    "SHVS": " SMART HYBRID ",
    "800DUO": "800 DUO",
    
    "^D 5|\sD 5": " D5 ",
    "MY 11": " MY11 ",
    "300 H": " 300H ",
    "4 MATIC": " 4MATIC ",
    "I TURBO": " TURBO ",
    "350 G": " 350G ",
    "200 K": " 200K ",
    "D 4D" : " 4D ",
    "AX 5" : " AX5 ",
    "R 30" : " R30 ",
    "AX 7" : " AX7 ",
    "E MAX": " MAX ",
    "DCT AT": " DCT ",
    "250 XL": " 250XL ",
    "250 XV": " 250XV ",
    "TI VCT": " TIVCT ",
    "TDI PD": " TDI ",
    "TDI CR": " TDI ",
    "ACPS" : " AC PS ",
    "30TDI": " 30 TDI ",
    "SDRIVE 20I ": " SDRIVE20I ",
    "XDRIVE 20D": " XDRIVE20D ",
    "SPORT LINE": " SPORTLINE ",
    "ALL STAR": " ALLSTAR ",
    r'DUAL TONE': " DUALTONE ",
    "LC200": " LC 200 ",
    "N10": " N 10 ",
    r'MHAWAK|M\sHAWK': " MHAWK ",
    "2 WD": "2WD",
    "4 WD": "4WD",
    "GATEWAY": "GETAWAY",
    "RHYTHM": "RHYTM",
    
    "GROOVE LIMITED EDITION": "GROOVE EDITION",    
    "SAFIRE90": "SAFIRE",
    
    "SMT": "S MT",
    "75PS": "75 PS",
    "90PS": "90 PS",
    r'QUADRAJET\s?90': "QUADRAJET 90 PS",
    
    
    r'^OP?T?\s+|\s+OP?T?$|\s+OP?T?\s+': " OPTIONAL ",
#     r'^OPT\s+|\s+OPT$|\s+OPT\s+': " OPTIONAL ",
    r'^OPTION\s+|\s+OPTION$|\s+OPTION\s+': " OPTIONAL ",
    r'^AUTO\s+|\s+AUTO$|\s+AUTO\s+': " AUTOMATIC ",
    r'^AM?T\s+|\s+AM?T$|\s+AM?T\s+|^AM?T$': " AUTOMATIC ",
#     r'^AMT\s+|\s+AMT$|\s+AMT\s+|^AMT$': " AUTOMATIC ",
    r'^DT\s+|\s+DT$|\s+DT\s+': " DUALTONE ",
    r'^MT\s+|\s+MT$|\s+MT\s+|^MT$': " MANUAL ",
    "TECHNOLOGY PACK": "TECHNOLOGY",
    
    r'^STD\s+|\s+STD$|\s+STD\s+|^STD|STD$': " STANDARD ",
    
    r"AEANKW|TRANSMISSION|TRANS|WITH|KIA SELTOS|EDITION|EDN|EDTION|KICKS|FLYING SPUR|BASE|LITRE|SINGLE|DSL|2ND|CAPTAIN|'|/": "",
    r'VAN|BE|W204|W 210|CBU|STANDARD|MPT|\sROOF\s|NEW|OLD|K SERIES |1L|TRD|\*|PACK|SEATER': "",
    r'333PS$|DPE|\sJTD\s16\sV|\d{2,3}\s?KW|CLASS$|BS III|BS\s?VI|BS4|BS\s?IV|BS6|GRADE|\(?BASE\sPRICE\)?|2022|2018' : ''
}

# PACK

for key in MAKE_CHANGE:
    other_sheet["A"] = other_sheet["A"].replace( key, MAKE_CHANGE[key], regex=True )
    cf_sheet["A"] = cf_sheet["A"].replace( key, MAKE_CHANGE[key], regex=True )

for key in MODEL_CHANGE:
    other_sheet["B"] = other_sheet["B"].replace( key, MODEL_CHANGE[key], regex=True )
    cf_sheet["B"] = cf_sheet["B"].replace( key, MODEL_CHANGE[key], regex=True )

for key in VARIANT_CHANGE:
    other_sheet["C"] = other_sheet["C"].replace( key, VARIANT_CHANGE[key], regex=True )
    cf_sheet["C"] = cf_sheet["C"].replace( key, VARIANT_CHANGE[key], regex=True )

other_sheet["D"] = other_sheet["D"].replace( "LPG", "CNG", regex=True )
cf_sheet["D"] = cf_sheet["D"].replace( "INTERNAL_LPG_CNG", "CNG", regex=True )

In [33]:
TRANSMISSION = [ 'MANUAL', 'AUTOMATIC', 'CVT', 'DCT', 'DCA', 'AMT', 'IMT', "TC", "IVT" ]

DRIVE = [ "AWD", "2WD", "4WD", "FWD" ]

ENGINE = { 
    "AUDI": [ "QUATTRO", "TFSI"  "TDI", "W12" ],
    "CHEVROLET": [ "TCDI", "TDCI" ],
    "FIAT": [ "FIRE"  "MULTIJET" ],
    "FORD": [ "TIVCT", "DURATEC", "ECOBOOST"  "TDCI", "DURATORQ", ],
    "HYUNDAI": [ "VTVT", "VTEC", "CRDI", "MPI", "KAPPA" ],
    "MAHINDRA": [ "MFALCON D75", "MFALCON D80", "MDI", "CRDE", "CRDI", "MFALCON G80" ],
    "MARUTI": [ "DECA" ],
    "MERCEDES BENZ": [ "CGI", "V8 RHD"  "CDI" ],
    "NISSAN": [ "110 PS" ],
    "PORSCHE": [ "PDK" ],
    "RENAULT": [ "99 PS", "101 PS", "104 PS"  "85 PS", "86 PS", "104 PS", "106 PS", "110 PS" ],
    "SKODA": [ "TSI", "MPI", "FSI", "TFSI"  "TDI", "CRDI" ],
    "TATA": [
        "CR4", "DICOR", "TDI", "TCIC", "REVOTRON 90 PS", "REVOTRON", "SAFIRE 90",  "REVOTORQ", "QUADRAJET 90 PS", "QUADRAJET 75 PS", "QUADRAJET90", "QUADRAJET 90", "QUADRAJET",
        "TURBO KRYOTEC", 
    ],  
    "TOYOTA": [ "D4D", "4D", "SPORTIVO" ],
    "VOLKSWAGEN": [ "TSI", "MPI", "TDI", "CR TDI", "CRDI" ],
    "VOLVO": [ "D3", "D4", "D5" ],
    "HONDA": [ "I VTEC", "I DTEC", "IVTEC", "IDTEC" ],
}

In [34]:
# cf_sheet["A"].unique()
# other_sheet["A"].unique()
# cf_sheet
# other_sheet

In [35]:
"""all makes in both master"""
makes_df = pd.DataFrame(columns=["CF", "DIGIT"])
makes = set()
cf_makes = set()
other_makes = set()

for index, row in cf_sheet.iterrows():
    cf_makes.add( str(row["A"]) )
    makes.add( str(row["A"]) )
    
for index, row in other_sheet.iterrows():
    other_makes.add( str(row["A"]) )
    makes.add( str(row["A"]) )
    
makes = sorted( makes )

"""matching of makes of both cf_master and pb_master"""

for make in makes:
    makes_df.loc[len(makes_df.index)] = [
        make if make in cf_makes else "",
        make if make in other_makes else ""
    ]

In [36]:
# makes_df

In [37]:
"""All models in both masters"""

models_df = pd.DataFrame(columns=["Make", "CF", "PB"])
models = {}
cf_models = {}
other_models = {}

for make in makes:
    model_set = set()
    cf_model_set = set()
    other_model_set = set()
    cf_models_list = cf_sheet[cf_sheet["A"] == make].filter("B")
    other_models_list = other_sheet[other_sheet[ "A" ] == make].filter("B")
    
    """adding all models of make in cf-master to models and cf_models set"""
    for i in range( len(cf_models_list) ):
        model_set.add( str(cf_models_list.iloc[i]["B"]) )
        cf_model_set.add( str(cf_models_list.iloc[i]["B"]) )
    
    """adding all models of make in acko-master to models and pb_models set"""
    for i in range( len(other_models_list) ):
        model_set.add( str(other_models_list.iloc[i]["B"]) )
        other_model_set.add( str(other_models_list.iloc[i]["B"]) )
    
    models[make] = model_set
    cf_models[make] = cf_model_set
    other_models[make] = other_model_set

"""matching of makes of both cf_master and pb_master"""

for make in makes:
    for model in sorted( models[ make ] ):
        models_df.loc[len(models_df.index)] = [
            make,
            model if model in cf_models[make] else "",
            model if model in other_models[make] else ""   
        ]

In [38]:
# models_df

In [39]:
""" List of make->model->variant and marking as matched/unmatched (True/False) """

CF_MMV = {}
OTHER_MMV = {}

for make in sorted(makes):
    """Adding make to the make-model-variant list"""
    
    CF_MMV[make] = {}
    OTHER_MMV[make] = {}
    
    for model in sorted( models[make] ):
        
        """Add model to the make in make->model->variant list"""
        CF_MMV[make][model] = {}
        OTHER_MMV[make][model] = {}
        
        """Filter all variant of make and model of both masters"""
        cf_variants = cf_sheet[ (cf_sheet["A"] == make) & (cf_sheet["B"] == model) ]
        other_variants = other_sheet[ (other_sheet["A"] == make) & (other_sheet["B"] == model) ]
    
        for index, row in cf_variants.iterrows():
            CF_MMV[make][model][ index ] = "NOT MATCHED"
            
        for index, row in other_variants.iterrows():
            OTHER_MMV[make][model][ index ] = "NOT MATCHED"

In [40]:
# CF_MMV = {}
# OTHER_MMV = {}

In [41]:
def get_matching_keys( make, VARIANT ):
    global TRANSMISSION
    global DRIVE
    global ENGINE
    
    key_set = set()
    
    for transmission in TRANSMISSION:
        if transmission in VARIANT:
            key_set = key_set | set( word_tokenize( transmission ) )
            
    for drive in DRIVE:
        if drive in VARIANT:
            key_set = key_set | set( word_tokenize( drive ) )

    if make in ENGINE:
        for engine in ENGINE[ make ]:
            if set( word_tokenize( engine)).issubset(VARIANT):
                key_set = key_set | set( word_tokenize( engine ) )
                                
    return key_set

def set_matched( cf_index, other_index ):
    
    global CF_MMV, OTHER_MMV
    
    CF_MMV[make][model][ cf_index ] = "MATCHED"
    OTHER_MMV[make][model][ other_index ] = "MATCHED"
    

def add_to_matched_list( matched_variant_df, cf_index, other_index, keys, status ):

    global OTHER_MMV
    global cf_sheet, cf_sheet_non_edited
    global other_sheet, other_sheet_non_edited
    
    cf_row = cf_sheet_non_edited.loc[ cf_index ]
    other_row = other_sheet_non_edited.loc[ other_index ]
    
    if OTHER_MMV[make][model][ other_index ] == "MATCHED":
        status = "REPEAT MATCHED"
    
    matched_variant_df.loc[len(matched_variant_df.index)] = [
        cf_row["A"],     #make
        cf_row["B"],     #model
        cf_row["CC"],    #cf_id
        other_row["CC"],    #pb_id
        f'{cf_row["A"]} {cf_row["B"]} {cf_row["C"]} {cf_row["D"]} {cf_row["E"]}',      #cf_variants_name
        f'{other_row["A"]} {other_row["B"]} {other_row["C"]} {other_row["D"]} {other_row["E"]}',      #pb_variants_name
        keys,            #matching_keys
        status           #status
    ]
    set_matched( cf_index, other_index )
    print( f'( {cf_row["CC"]} ) {cf_row["C"]} {cf_row["D"]} {cf_row["E"]} \t', cf_variant, '********* \t\t\t\t\tMATCHED' )
    
def check_CC_FUEL_SEAT(cf_row, pb_row):
    return (cf_row["D"] == other_row["D"]) and (int(other_row["E"]) == int(cf_row["E"])) and (cf_row["F"] == other_row["F"])

def check_CC_FUEL_DIFF_SEAT(cf_row, pb_row):
    return (cf_row["D"] == other_row["D"]) and (abs(int(other_row["E"]) - int(cf_row["E"])) <= 50) and (cf_row["F"] == other_row["F"])
    
def check_CC_FUEL(cf_row, pb_row):
    return (cf_row["D"] == other_row["D"]) and (int(other_row["E"]) == int(cf_row["E"]))

In [1]:
"""mapping variants of pb master with cf master"""
matched_variant_df = pd.DataFrame(columns=["Make", "Model", "CF_ID", "OTHER_ID", "CF_Variant", "OTHER_Variant", "Keys", "Status"])

""" count of matched variants """
count = 0   

for make in sorted(cf_makes):
    for model in sorted( models[make] ):        
        
        """Filter all variant of make and model of both masters"""
        cf_variants = cf_sheet[ (cf_sheet["A"] == make) & (cf_sheet["B"] == model) ].sort_values('C')
        other_variants = other_sheet[ (other_sheet["A"] == make) & (other_sheet["B"] == model) ].sort_values('C')
         
        for other_index, other_row in other_variants.iterrows():
            
            other_variant = set( word_tokenize( f'{other_row["B"]} {other_row["C"]}' )) - {"P", "D", "DIESEL", "PETROL", "CNG"}
            other_key = get_matching_keys(make, other_variant ) 

            print(f'\n\n{other_row["CC"]} {other_row["A"]} {other_row["B"]} \n{other_row["C"]} {other_row["D"]} {other_row["E"]}', other_variant )
            print( "Matching key ", other_key, "\n\n" )
            
            for cf_index, cf_row in cf_variants.iterrows():
                
                if other_row["D"] == "ELECTRIC" and other_row["F"] == cf_row["F"]:
                    
                    cf_variant = set( word_tokenize( f'{cf_row["B"]} {cf_row["C"]}' ) ) - {"P", "D", "DIESEL", "PETROL", "CNG"}
                    cf_key = get_matching_keys(make, cf_variant )

                    if cf_variant == other_variant:
                        count += 1
                        add_to_matched_list( matched_variant_df, cf_index, other_index, {}, "MATCHED ELECTRIC" )

                elif check_CC_FUEL_SEAT( cf_row, other_row ):
#                 if check_CC_FUEL( cf_row, other_row ):         #if seater column is not present

                        cf_variant = set( word_tokenize( f'{cf_row["B"]} {cf_row["C"]}' ) ) - {"P", "D", "DIESEL", "PETROL", "CNG"}
                        cf_key = get_matching_keys(make, cf_variant )

                        if cf_variant == other_variant:
                            count += 1
                            add_to_matched_list( matched_variant_df, cf_index, other_index, {}, "MATCHED" )
                                                    
#                         elif cf_variant | other_key == other_variant :
#                             count += 1
#                             add_to_matched_list( matched_variant_df, cf_index, other_index, other_key , "MATCHED PB_KEYS" )
                                    
#                         elif cf_variant == other_variant | cf_key:
#                             count += 1
#                             add_to_matched_list( matched_variant_df, cf_index, other_index, cf_key , "MATCHED CF_KEYS" )
                                                    
                        else:
                            print( f'( {cf_row["CC"]} ) {cf_row["C"]} {cf_row["D"]} {cf_row["E"]}\t', cf_variant )
                    
count

In [2]:
"""Matching using additional keys"""

for make in sorted(cf_makes):
    for model in sorted( models[make] ):        
        
        """Filter all variant of make and model of both masters"""
        cf_variants = cf_sheet[ (cf_sheet["A"] == make) & (cf_sheet["B"] == model) ].sort_values('C')
        other_variants = other_sheet[ (other_sheet["A"] == make) & (other_sheet["B"] == model) ].sort_values('C')
         
        for other_index, other_row in other_variants.iterrows():
            
            """Skip matched variants"""
            if OTHER_MMV[make][model][ other_index ] == "MATCHED":
                continue
            
            other_variant = set( word_tokenize( f'{other_row["B"]} {other_row["C"]}' )) - {"P", "D", "DIESEL", "PETROL", "CNG"}
            other_key = get_matching_keys(make, other_variant ) 

            print(f'\n\n{other_row["CC"]} {other_row["A"]} {other_row["B"]} \n{other_row["C"]} {other_row["D"]} {other_row["E"]}', other_variant )
            print( "Matching key ", other_key, "\n\n" )
            
            for cf_index, cf_row in cf_variants.iterrows():
                
                """Skip matched variants"""
                if CF_MMV[make][model][ cf_index ] == "MATCHED":
                    continue
                
                if check_CC_FUEL_SEAT( cf_row, other_row ):
#                 if check_CC_FUEL( cf_row, other_row ):         #if seater column is not present

                        cf_variant = set( word_tokenize( f'{cf_row["B"]} {cf_row["C"]}' ) ) - {"P", "D", "DIESEL", "PETROL", "CNG"}
                        cf_key = get_matching_keys(make, cf_variant )
                        
#                         if cf_variant == other_variant:
#                             count += 1
#                             add_to_matched_list( matched_variant_df, cf_index, other_index, {}, "MATCHED" )
                                                    
                        if cf_variant | other_key == other_variant :
                            count += 1
                            add_to_matched_list( matched_variant_df, cf_index, other_index, other_key , "MATCHED PB_KEYS" )
                                    
                        elif cf_variant == other_variant | cf_key:
                            count += 1
                            add_to_matched_list( matched_variant_df, cf_index, other_index, cf_key , "MATCHED CF_KEYS" )
                                                    
                        else:
                            print( f'( {cf_row["CC"]} ) {cf_row["C"]} {cf_row["D"]} {cf_row["E"]}\t', cf_variant )
                    
count

In [3]:
"""Matching CC difference"""

for make in sorted(cf_makes):
    for model in sorted( models[make] ):        
        
        """Filter all variant of make and model of both masters"""
        cf_variants = cf_sheet[ (cf_sheet["A"] == make) & (cf_sheet["B"] == model) ].sort_values('C')
        other_variants = other_sheet[ (other_sheet["A"] == make) & (other_sheet["B"] == model) ].sort_values('C')
         
        for other_index, other_row in other_variants.iterrows():
            
            """Skip matched variants"""
            if OTHER_MMV[make][model][ other_index ] == "MATCHED":
                continue
            
            other_variant = set( word_tokenize( f'{other_row["B"]} {other_row["C"]}' )) - {"P", "D", "DIESEL", "PETROL", "CNG"}
            other_key = get_matching_keys(make, other_variant ) 

            print(f'\n\n{other_row["CC"]} {other_row["A"]} {other_row["B"]} \n{other_row["C"]} {other_row["D"]} {other_row["E"]}', other_variant )
            print( "Matching key ", other_key, "\n\n" )
            
            for cf_index, cf_row in cf_variants.iterrows():
                
                """Skip matched variants"""
                if CF_MMV[make][model][ cf_index ] == "MATCHED":
                    continue

                if check_CC_FUEL_DIFF_SEAT( cf_row, other_row ):
#                 if check_CC_FUEL( cf_row, other_row ):         #if seater column is not present

                        cf_variant = set( word_tokenize( f'{cf_row["B"]} {cf_row["C"]}' ) ) - {"P", "D", "DIESEL", "PETROL", "CNG"}
                        cf_key = get_matching_keys(make, cf_variant )
                        
#                         if cf_variant == other_variant:
#                             count += 1
#                             add_to_matched_list( matched_variant_df, cf_index, other_index, {}, "MATCHED" )
                                                    
                        if cf_variant | other_key == other_variant :
                            count += 1
                            add_to_matched_list( matched_variant_df, cf_index, other_index, other_key , "MATCHED CC_DIFF" )
                                    
                        elif cf_variant == other_variant | cf_key:
                            count += 1
                            add_to_matched_list( matched_variant_df, cf_index, other_index, cf_key , "MATCHED CC_DIFF" )
                                                    
                        else:
                            print( f'( {cf_row["CC"]} ) {cf_row["C"]} {cf_row["D"]} {cf_row["E"]}\t', cf_variant )
                    
count

In [4]:
count

In [46]:
# cf_variant_df.shape[0]

In [47]:
cf_variant_df = pd.DataFrame(columns=["ID", "Make", "Model", "Variant", "Status"])

for make in sorted(cf_makes):
    for model in sorted(models[make]):
        for variant_id in CF_MMV[make][model]:
            cf_variant_df.loc[ len(cf_variant_df.index) ] = [
                cf_sheet_non_edited.loc[ variant_id ]["CC"],
                make, 
                model,
                f'{cf_sheet_non_edited.loc[ variant_id ]["C"]} {cf_sheet_non_edited.loc[ variant_id ]["D"]} {cf_sheet_non_edited.loc[ variant_id ]["E"]} ({cf_sheet_non_edited.loc[ variant_id ]["F"]}Str)',
                CF_MMV[make][model][variant_id]
            ]

In [48]:
other_variant_df = pd.DataFrame(columns=["ID", "Make", "Model", "Variant", "Status"])

for make in sorted(cf_makes):
    for model in sorted(models[make]):
        for variant_id in sorted(OTHER_MMV[make][model]):
            other_variant_df.loc[ len(other_variant_df.index) ] = [
                other_sheet_non_edited.loc[ variant_id ]["CC"],
                make, 
                model, 
                f'{other_sheet_non_edited.loc[ variant_id ]["C"]} {other_sheet_non_edited.loc[ variant_id ]["D"]} {other_sheet_non_edited.loc[ variant_id ]["E"]} ({other_sheet_non_edited.loc[ variant_id ]["F"]}Str)', 
                OTHER_MMV[make][model][variant_id]
            ]

In [49]:
unmatched_variant_df = pd.DataFrame(columns=["Make", "Model","CF_ID", "OTHER_ID", "CF_Variant", "OTHER_Variant"])

for make in sorted(makes):
    for model in sorted(models[make]):
        
        cf_variants = cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Model"] == model) & (cf_variant_df["Status"] == "NOT MATCHED") ]
        other_variants = other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Model"] == model) & (other_variant_df["Status"] == "NOT MATCHED") ]
        
        row_count = max( cf_variants.shape[0], other_variants.shape[0] )
        
        for i in range( row_count ):
            
            unmatched_variant_df.loc[ len(unmatched_variant_df.index) ] = [
                make,
                model,
                cf_variants.iloc[i]["ID"] if i < cf_variants.shape[0] else None,
                other_variants.iloc[i]["ID"] if i < other_variants.shape[0] else None,
                cf_variants.iloc[i]["Variant"] if i < cf_variants.shape[0] else None,
                other_variants.iloc[i]["Variant"] if i < other_variants.shape[0] else None
            ]

In [6]:
summary_make_df = pd.DataFrame(columns=["Make", "CF_Total", "CF_Matched", "CF_Unamtched", "Acko_Total", "Acko_Matched",  "Acko_Unmatched"])

sum_pb = sum_cf = 0
for make in makes:
    summary_make_df.loc[ len(summary_make_df.index) ] = [
        make,
        cf_variant_df[ (cf_variant_df["Make"] == make) ].shape[0],
        cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Status"] == "MATCHED") ].shape[0],
        cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Status"] == "NOT MATCHED") ].shape[0],
        other_variant_df[ (other_variant_df["Make"] == make) ].shape[0],
        other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Status"] == "MATCHED") ].shape[0],
        other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Status"] == "NOT MATCHED") ].shape[0]
    ]
    
# summary_make_df

In [51]:
summary_model_df = pd.DataFrame(columns=["Make", "Model", "CF_Total", "CF_Matched", "CF_Unamtched", "Acko_Total", "Acko_Matched",  "Acko_Unmatched"])

sum_pb = sum_cf = 0
for make in makes:
    for model in models[make]:
        summary_model_df.loc[ len(summary_model_df.index) ] = [
            make,
            model,
            cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Model"] == model) ].shape[0],
            cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Model"] == model) & (cf_variant_df["Status"] == "MATCHED") ].shape[0],
            cf_variant_df[ (cf_variant_df["Make"] == make) & (cf_variant_df["Model"] == model) & (cf_variant_df["Status"] == "NOT MATCHED") ].shape[0],
            other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Model"] == model) ].shape[0],
            other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Model"] == model) & (other_variant_df["Status"] == "MATCHED") ].shape[0],
            other_variant_df[ (other_variant_df["Make"] == make) & (other_variant_df["Model"] == model) & (other_variant_df["Status"] == "NOT MATCHED") ].shape[0]
        ]

In [52]:
with pd.ExcelWriter('./CF_ACKO.xlsx') as writer:
    makes_df.to_excel( writer, sheet_name="Makes", index=False )
    models_df.to_excel( writer, sheet_name="Models", index=False )
    matched_variant_df.to_excel( writer, sheet_name="Matched", index=False )
    unmatched_variant_df.to_excel( writer, sheet_name="Unmatched", index=False )
    cf_variant_df.to_excel( writer, sheet_name="CF_Variants", index=False )
    other_variant_df.to_excel( writer, sheet_name="OTHER_Variants", index=False )
    summary_make_df.to_excel( writer, sheet_name="Summary Make", index=False )
    summary_model_df.to_excel( writer, sheet_name="Summary Model", index=False )  